This is [Mark Ponder's example code](github.com/ponde006/BLP_Code/blob/main/BLP%20Replication.ipynb).

In [1]:
using DataFrames
using DataFramesMeta
using LinearAlgebra
using CSV # Read Data
using BenchmarkTools
using NLopt
using Distributions

# For Halton Draws
using StatsFuns
using Primes
using Statistics
#using Sobol

#cars = DataFrame!(CSV.File("[filepath]/Automobile Data.csv", header = true));
cars = DataFrame!(CSV.File("C:/Users/ponde/Dropbox/Nonparametric Random Coefficients/Raw Data/Automobile Data.csv", header = true));

In [2]:
# Estimated means and deviations for lognormal distribution
incomeMeans = [2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745,
2.09805, 2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426,
2.18071, 2.18856, 2.21250, 2.18377]
 
sigma_v = 1.72

1.72

In [3]:
βs = [-7.061, 2.883, 1.521, -0.122, 3.46]
γs = [0.952, 0.477, 0.619, -0.415, -0.046, 0.019]
α = 43.501
σs = [3.612, 4.628, 1.818, 1.050, 2.056];

In [4]:
cars[!,:ln_hpwt] = log.(cars[!,:hpwt]);
cars[!,:ln_space] = log.(cars[!,:space])
cars[!,:ln_mpg] = log.(cars[!,:mpg])
cars[!,:ln_mpd] = log.(cars[!,:mpd])
cars[!,:ln_price] = log.(cars[!,:price])
cars[!,:trend] = cars[!,:market] .- 1 
cars[!,:cons] = ones(size(cars,1));

In [5]:
regSet = @linq cars |>
    by(:model_year, s_0 = log.(1 - sum(:share)))
 
regSet = innerjoin(cars, regSet, on = :model_year);
  
regSet = @linq regSet |>
    transform(s_i = log.(:share)) |>
    transform(dif = :s_i - :s_0);

regSet[!,:dif_2] = log.(regSet[!,:share]) .- log.(regSet[!,:share_out]) ;
regSet[!,:ln_price] = log.(regSet[!,:price]);
 
regSet = sort!(regSet, [:market,  :firmid])
 
markets = convert(Array{Int64,2} , regSet[:, [ :market] ])[:]
marks = unique(markets)
firms = convert(Matrix, regSet[:, [ :firmid] ]);
modlist = unique(regSet[!,:newmodv])
modLoc = regSet[!,:newmodv];

In [6]:
X = convert(Array{Float64,2}, regSet[:, [ :cons, :hpwt, :air, :mpd, :space] ]) # Demand Variables
W = convert(Array{Float64,2}, regSet[:, [ :cons, :ln_hpwt, :air, :ln_mpg, :ln_space, :trend] ]) # Supply Variables
price = convert(Array{Float64,2}, regSet[:, [ :price] ]) # Price
delta_0 = convert(Array, round.(regSet[:,:dif_2], digits = 20));

In [7]:
J_f = Array{Int64}(undef, 0,2)
count = 0
for ms = 1:20
    midx = findall(markets .== ms)
    tfirms = unique(firms[midx])
    for f in tfirms
        J_f = vcat(J_f, [count + findfirst(firms[midx] .== f) count + findlast(firms[midx] .== f)])
    end
    count += length(midx)
end;

In [8]:
# Beginning and end of the market ids
idxs = [findfirst(markets .== i) for i = 1:20]
idxe = [findlast(markets .== i) for i = 1:20]
midx = hcat(idxs, idxe);

## Instrumental Variable Construction

In [9]:
# Neighborhood Distance Instruments
function gen_gh_inst_1( inX, midx )

    my_insts = 0
    # Loop over markets
    for m in 1:size(midx,1)
        # find input obs
        sub = inX[midx[m,1]:midx[m,2],:]
        # calculate standard deviation
        stdx = std(sub, dims = 1)
        # Initialize empty array
        z_1 = Array{Float64,2}(undef, size(sub,1), 0)
        
        # Loop over observations, calculating differences
        for i = 1:size(inX,2)
            unique_vals = unique(sub[:,i])
            # Correct for indicator variables
            if (unique_vals == [0.0, 1.0]) | (unique_vals == [1.0]) | (unique_vals == [0.0]) | (unique_vals == [1.0, 0.0])
                z_1 = hcat(z_1, sum(abs.(sub[:,i] .!= sub[:,i]'), dims = 2))
            else
                z_1 = hcat(z_1, sum((abs.(sub[:,i] .- sub[:,i]') .< stdx[i]), dims = 2))
            end
            # Calculate the interactions
            for j = 1:size(inX,2)
                # Correct for indicator variables
                if (unique_vals == [0.0, 1.0]) | (unique_vals == [1.0]) | (unique_vals == [0.0]) | (unique_vals == [1.0, 0.0])
                    z_1 = hcat(z_1, sum(abs.(sub[:,i] .!= sub[:,i]').*(sub[:,j] .- sub[:,j]'), dims = 2))
                else
                    z_1 = hcat(z_1, sum((abs.(sub[:,i] .- sub[:,i]') .< stdx[i]).*(sub[:,j] .- sub[:,j]'), dims = 2))
                end
            end
        end
            
        if m == 1   
            my_insts = z_1
        else
            my_insts = vcat(my_insts, z_1)
        end
    end

    return my_insts
end

gen_gh_inst_1 (generic function with 1 method)

In [10]:
phat = W*((W'W)\W'log.(price))
Zgh = gen_gh_inst_1(hcat(X[:,2:end], exp.(phat)), midx )
(Zgh'Zgh)\(Zgh'Zgh)

30×30 Array{Float64,2}:
  1.0           1.27595e-17   7.68445e-17  …   1.3791e-16    1.02269e-15
 -1.19217e-14   1.0           1.84779e-14     -1.63924e-15  -7.99009e-14
 -6.03433e-15  -6.3772e-17    1.0             -9.59052e-16  -2.48466e-14
  5.29915e-16  -1.11265e-17  -2.39223e-15      1.82591e-15   6.04439e-15
 -8.50387e-16  -1.05383e-16   5.52108e-15      4.58036e-15  -7.88542e-15
  6.72394e-16   2.72263e-18  -1.14914e-15  …   8.00091e-17   2.38996e-15
  4.40258e-16   1.22282e-17   3.31554e-17     -1.01952e-16  -1.60014e-16
  2.6964e-14   -5.98333e-16  -2.86341e-14     -7.7746e-15   -1.65858e-14
  1.46121e-14  -1.60566e-16  -3.75517e-14     -2.92485e-15   3.18525e-14
 -3.00873e-15  -3.68084e-17   4.45555e-15      2.40802e-16  -1.1457e-15
  5.29071e-15  -3.46808e-17   1.56639e-15  …  -3.91103e-15  -2.19148e-14
 -1.54849e-15   1.59514e-17   3.08783e-15      3.72826e-16  -1.66471e-15
 -6.2113e-15    3.36438e-17  -1.53046e-15     -2.91253e-17   5.95349e-15
  ⋮                         

In [11]:
# Quadratic Distance Instruments
function gen_gh_inst_2( inX, midx )

    my_insts = 0
    # Loop over markets
    for m in 1:size(midx,1)
        # find input obs
        sub = inX[midx[m,1]:midx[m,2],:]
        # calculate standard deviation
        stdx = std(sub, dims = 1)
        # Initialize empty array
        z_1 = Array{Float64,2}(undef, size(sub,1), 0)
        
        # Loop over observations, calculating differences
        for i = 1:size(inX,2)
            unique_vals = unique(sub[:,i])
            # Correct for indicator variables
            if (unique_vals == [0.0, 1.0]) | (unique_vals == [1.0]) | (unique_vals == [0.0]) | (unique_vals == [1.0, 0.0])
                z_1 = hcat(z_1, sum((sub[:,i] .!= sub[:,i]'), dims = 2))
            else
                z_1 = hcat(z_1, sum((sub[:,i] .- sub[:,i]').^2, dims = 2))
            end
            # Calculate the interactions
            for j = i+1:size(inX,2)
                # Correct for indicator variables
                if (unique_vals == [0.0, 1.0]) | (unique_vals == [1.0]) | (unique_vals == [0.0]) | (unique_vals == [1.0, 0.0])
                    z_1 = hcat(z_1, sum((sub[:,i] .!= sub[:,i]').*(sub[:,j] .- sub[:,j]'), dims = 2))
                else
                    z_1 = hcat(z_1, sum((sub[:,i] .- sub[:,i]').*(sub[:,j] .- sub[:,j]'), dims = 2))
                end
            end
        end
            
        if m == 1   
            my_insts = z_1
        else
            my_insts = vcat(my_insts, z_1)
        end
    end

    return my_insts
end

gen_gh_inst_2 (generic function with 1 method)

In [12]:
phat = W*((W'W)\W'log.(price))
Zgh = gen_gh_inst_2(hcat(X[:,2:end], exp.(phat)), midx )
Zgh

2217×15 Array{Float64,2}:
 3.37361   0.0        0.111778   4.66214   …  12.1038    65.7027    461.874
 2.62967   0.0       -4.67432    5.49612      23.7214   117.744     690.554
 2.01381   0.0       -2.00414    2.84391      23.5562    97.4828    525.278
 2.16614   0.0       -2.97013    3.71751      19.0519    90.5837    539.694
 2.33019   0.0       -5.16327    5.10697      30.8042   143.542     778.178
 2.61201   0.0       -8.09451    6.40681   …  34.7873   175.521     991.0
 2.45972   0.0       -5.08303    5.31094      26.9796   130.357     737.136
 8.52447   0.0      -24.0638    16.6845       39.6723   282.545    2152.21
 2.49452   0.0       -5.70538    5.29426      25.47     130.936     778.133
 2.43432   0.0       -2.5334     3.50397      11.2181    63.7938    472.496
 3.65033   0.0       -4.78246    4.59545   …  11.2871    77.5673    716.601
 2.18024   0.0       -2.63822    3.72085      18.2625    84.0745    499.035
 2.04489   0.0       -1.60437    2.74719      11.2871    53.4691 

In [13]:
# BLP Instruments
function gen_inst( inX, normal = 1 )
    totMarket = similar(inX)
    totFirm = similar(inX)

    for m in marks
        sub = inX[findall(markets .== m),:]
        firminfo = firms[findall(markets .== m),:]
        #modelinfo = modLoc[find(markets .== m),:]
        sameFirm = convert(Array{Float64,2}, firminfo .== firminfo')
        #sameFirm = sameFirm - diagm(diag(sameFirm))

        #sameProduct = ones(sameFirm) - diagm(ones(size(sub,1), 1)[:])
        z_1 = similar(sub)
        for i = 1:size(sub, 2)
            if normal == 1
                z_1[:,i] = sum((sub[:,i] .* sameFirm), dims = 1)' # Prime on sameFirm is key
            else
                z_1[:,i] = sum((sub[:,i] .* (sameFirm - I))', dims = 1)' # Prime on sameFirm is key
            end
        end
        totFirm[findall(markets .== m),:] = z_1

        # Within Market
        sub = inX[findall(markets .== m),:]
        z_1 = similar(sub)
        sameFirm = firminfo .== firminfo'
        for i = 1:size(sub, 2)
            if normal == 1
                z_1[:,i] = sum(sub[:,i] .* (sameFirm .+ .!sameFirm ), dims = 1)
            else
                z_1[:,i] = sum((sub[:,i] .* (.!sameFirm )), dims = 1)
            end
        end
        totMarket[findall(markets .== m),:] = z_1
    end

    return [totFirm, totMarket]
end

gen_inst (generic function with 2 methods)

In [14]:
Z_x_1, Z_x_2 = gen_inst(X,0)
Z_w_1, Z_w_2 = gen_inst(W,0)
Z_x = hcat(X, Z_x_1, Z_x_2)
Z_w = hcat(W, Z_w_1, Z_w_2, convert(Array{Float64,1}, regSet[:mpd]))
Z_ = [Z_x zeros(size(Z_w)); zeros(size(Z_x)) Z_w];

In [15]:
mutable struct ModelData
    price::Array{Float64, 1}
    X::Array{Float64, 2}
    alpha::Float64
    beta::Array{Float64, 1}
    gamma::Array{Float64, 1}
    sigma::Array{Float64, 1}
    delta::Array{Float64, 1}
    xi::Array{Float64, 1}
    marketidx::Array{Int64, 1} # Market IDs in a vector
    midx::Array{Int64, 2} # Beginning and end index for each market
    J_f::Array{Int64, 2} # Beginning and end index for each firm's products
    y_it::Array{Float64, 2} # Random income draws
    v_ik::Array{Float64, 2} # Random coefficient draws
    unobs_weight::Array{Float64, 1}
    p_ijt::Array{Float64, 2} # Array for individual choice probability
    s_jt::Array{Float64, 1} # Array for implied market share
    act_s::Array{Float64, 1} # Array for actual market shares
    mc::Array{Float64, 1}  # Array for marginal costs
    optinst::Array{Float64,2} # Array of Optimal Instruments
    W::Array{Float64, 2}
    
    function ModelData( price , X , alpha , beta , gamma, sigma, delta, xi, marketidx, midx, J_f, y_it, v_ik,
                        unobs_weight, p_ijt, s_jt, act_s, mc, optinst)
        return new( price , X , alpha , beta , gamma, sigma, delta, xi, marketidx, midx, J_f, y_it, v_ik,
                        unobs_weight, p_ijt, s_jt, act_s, mc, optinst, zeros(size(X)) )
    end
end

In [16]:
ns = 1000*20
v_ik1 = randn(6,  ns )'
m_t = repeat(incomeMeans, inner = [ns, 1])
  
#y_it = exp(m_t + sigma_v * repeat(v_ik[:,end], outer = [length(incomeMeans),1]));
y_it1 = exp.(incomeMeans .+ sigma_v * v_ik1[:,end]');
 
unobs_weight = ones(ns)'/(ns)*20;

In [17]:
function gen_mm2(m::ModelData)
    α = m.alpha
    v_ik = m.v_ik
    J_f = m.J_f
    midx = m.midx
    for i = 1:length(m.p_ijt)
        m.p_ijt[i] = 0.0
    end
  
    m.delta .= exp.(m.delta)
        
    step = Int(size(m.v_ik,1)/size(m.midx,1))
 
    # iterate over markets
    for mi = 1:size(m.midx,1)
        muidx1 = step*(mi - 1) + 1; muidx2 = step*mi
 
        eps = 1.0
        count = 1
 
        r1 = midx[mi,1]; r2 = midx[mi,2];
 
        # precalculate random coeffs
        mui = zeros(r2-r1+1, size(m.p_ijt,2)) # Allocate Array
        randcoeff = (m.sigma' .* m.v_ik[muidx1:muidx2,:])
        yp = m.y_it[mi,muidx1:muidx2]'.*m.price[r1:r2]
        # calculate random coefficient
        BLAS.gemm!('N', 'T', 1.0, m.X[r1:r2,:], randcoeff, 0.0, mui)
        mui .-= α*yp
        mui .= exp.(mui);
 
        marketmat = ones(r2-r1+1, r2-r1+1)
        # contraction mapping/fixed point
        while (eps > 1e-14)*(200 > count)
 
            numer = mui.*m.delta[r1:r2];
            s0 = 1.0./(1.0 .+ sum(numer,dims=1))
            m.p_ijt[r1:r2,:] .= numer .* s0;
 
            m.s_jt[r1:r2] .= m.p_ijt[r1:r2,:] * m.unobs_weight[muidx1:muidx2]
             
           # Finding the Fixed Point
            if eps > 0.1
                # Nevo Contraction
                m.delta[r1:r2] .= m.delta[r1:r2] .* m.act_s[r1:r2]./m.s_jt[r1:r2]
                eps = maximum(abs.((m.s_jt[r1:r2]./m.act_s[r1:r2]) .- 1))
     
            else
                # Newton Root-finding
                m.delta[r1:r2] .= log.(m.delta[r1:r2])
                tmp2 = m.p_ijt[r1:r2,:] .* m.unobs_weight[muidx1:muidx2]'
 
                Jf = I - (tmp2*m.p_ijt[r1:r2,:]')./m.s_jt[r1:r2,:]
                diff = log.(m.s_jt[r1:r2]) .- log.(m.act_s[r1:r2])
                stp = -Jf\diff
                m.delta[r1:r2,:] .= m.delta[r1:r2,:] .+ stp
                eps = maximum(abs.(stp))
                m.delta[r1:r2] .= exp.(m.delta[r1:r2])
            end
     
            count += 1
        end
 
    end
    m.delta .= log.(m.delta)
 
    # calculate mc
    for j = 1:size(J_f, 1)
         
        idx1 = J_f[j,1]; idx2 = J_f[j,2];
        muidx = searchsortedlast(m.midx[:,1], idx1 )
        muidx_1 = (muidx-1)*step + 1
        muidx_2 = (muidx)*step
        rp = m.p_ijt[idx1:idx2,:] 
        wα =  m.unobs_weight[muidx_1:muidx_2] .* m.y_it[muidx,muidx_1:muidx_2] * α
        nfirms = idx2 - idx1 + 1;
        tmp2 = rp .* wα'
        tmpvec = diagm(rp * wα);
        BLAS.gemm!('N', 'T', -1.0, m.p_ijt[idx1:idx2,:], tmp2 , 1.0, tmpvec) 
 
        b = tmpvec\m.s_jt[idx1:idx2]
        m.mc[idx1:idx2] .= m.price[idx1:idx2] .- b
    end
    for i = 1:length(m.mc)
        m.mc[i] = ifelse(m.mc[i] < 0, 0.001, m.mc[i])
    end
    return nothing
end

gen_mm2 (generic function with 1 method)

In [18]:
d_share = convert(Array{Float64,1}, regSet[:share])
m = ModelData(  price[:], X, α, βs, γs, σs, deepcopy(delta_0), deepcopy(delta_0), markets[:], midx, J_f, 1 ./y_it1, v_ik1[:,1:end-1], 
  unobs_weight[:], zeros(size(X,1), Int(size(v_ik1,1)/20)), convert(Array{Float64,1}, deepcopy(d_share)), 
    convert(Array{Float64,1}, deepcopy(d_share)), deepcopy(delta_0), zeros(size(X,1), 19));
 
@time gen_mm2(m)
mean(m.delta)

  1.177374 seconds (15.02 k allocations: 1.097 GiB, 9.86% gc time)


-0.7227089043692163

In [19]:
function GMM(param, m, wght)
    m.alpha = abs(param[1])
    m.sigma = abs.(param[2:end])
     
    gen_mm2(m)
    Z_ = m.optinst
    y_ = vcat(m.delta, log.(m.mc))
 
    X_ = [m.X zeros(size(m.W)); zeros(size(m.X)) m.W]
    zx = Z_'*X_
     
    βols = (zx'wght*zx)\(zx'wght*Z_'y_)
    ϵ = y_ - X_*βols
    g = Z_'*ϵ/length(ϵ)
    m.beta = βols[1:size(m.X,2)]
    m.gamma = βols[size(m.X,2)+1:end]
    res = g'wght*g*length(g)
  #  println(m.alpha, m.sigma, res)
    return res
end


GMM (generic function with 1 method)

In [ ]:
m.W = W
m.optinst = Z_;
 
tp = [α, σs...];
wght = inv(Z_'Z_);
 
y_ = vcat(m.delta, log.(m.mc))
X_ = [m.X zeros(size(m.W)); zeros(size(m.X)) m.W]
zx = Z_'*X_
 
βols = (zx'wght*zx)\(zx'wght*Z_'y_)
ϵ = y_ - X_*βols;
mm = Z_ .* ϵ
g = mean(mm, dims=1)
wght = inv(mm'mm/size(mm,1) .- g.*g');
 
opt = Opt(:LN_NELDERMEAD, 6)
initial_step!(opt, abs.(tp)*0.1 ) 
ftol_rel!(opt,1e-4)
xtol_rel!(opt,1e-4)
maxeval!(opt, 250)
min_objective!(opt, (x,y) -> GMM(x, m, wght))
@time minf_,minx_,ret_ = NLopt.optimize(opt, tp )